In [1]:
import awswrangler as wr

In [2]:
bucket = 'my-bucket-556-115-65'
path = f"s3://{bucket}/data/"

In [3]:
if "awswrangler_test" not in wr.catalog.databases().values:
    wr.catalog.create_database("awswrangler_test")

In [4]:
cols = ["id", "dt", "element", "value", "m_flag", "q_flag", "s_flag", "obs_time"]

df = wr.s3.read_csv(
    path="s3://noaa-ghcn-pds/csv/189",
    names=cols,
    parse_dates=["dt", "obs_time"])  # Read 10 files from the 1890 decade (~1GB)

df.head()

,id,dt,element,value,m_flag,q_flag,s_flag,obs_time
0,AGE00135039,1890-01-01,TMAX,160,NaN,NaN,E,NaN
1,AGE00135039,1890-01-01,TMIN,30,NaN,NaN,E,NaN
2,AGE00135039,1890-01-01,PRCP,45,NaN,NaN,E,NaN
3,AGE00147705,1890-01-01,TMAX,140,NaN,NaN,E,NaN
4,AGE00147705,1890-01-01,TMIN,74,NaN,NaN,E,NaN


In [5]:
wr.s3.to_parquet(
    df=df,
    path=path,
    dataset=True,
    mode="overwrite",
    database="awswrangler_test",
    table="noaa"
);

In [6]:
wr.catalog.table(database="awswrangler_test", table="noaa")

,Column Name,Type,Partition,Comment
0,id,string,False,
1,dt,timestamp,False,
2,element,string,False,
3,value,bigint,False,
4,m_flag,string,False,
5,q_flag,string,False,
6,s_flag,string,False,
7,obs_time,string,False,


In [8]:
%%time
# Reading with ctas_approach=False
wr.athena.read_sql_query("SELECT * FROM noaa", database="awswrangler_test", ctas_approach=False)

CPU times: user 1min 36s, sys: 5.49 s, total: 1min 41s
Wall time: 3min 53s


,id,dt,element,value,m_flag,q_flag,s_flag,obs_time
0,AGE00135039,1890-01-01,TMAX,160,<NA>,<NA>,E,<NA>
1,AGE00135039,1890-01-01,TMIN,30,<NA>,<NA>,E,<NA>
2,AGE00135039,1890-01-01,PRCP,45,<NA>,<NA>,E,<NA>
3,AGE00147705,1890-01-01,TMAX,140,<NA>,<NA>,E,<NA>
4,AGE00147705,1890-01-01,TMIN,74,<NA>,<NA>,E,<NA>
...,...,...,...,...,...,...,...,...
29567911,UZM00038457,1899-12-31,PRCP,16,<NA>,<NA>,r,<NA>
29567912,UZM00038457,1899-12-31,TAVG,-73,<NA>,<NA>,r,<NA>
29567913,UZM00038618,1899-12-31,TMIN,-76,<NA>,<NA>,r,<NA>
29567914,UZM00038618,1899-12-31,PRCP,0,<NA>,<NA>,r,<NA>


In [9]:
%%time
# Default with ctas_approach=True - 13x faster (default)
wr.athena.read_sql_query("SELECT * FROM noaa", database="awswrangler_test")

CPU times: user 47.2 s, sys: 11.4 s, total: 58.6 s
Wall time: 56.1 s


,id,dt,element,value,m_flag,q_flag,s_flag,obs_time
0,ASN00053001,1890-01-06,PRCP,0,<NA>,<NA>,a,<NA>
1,ASN00053005,1890-01-06,PRCP,0,<NA>,<NA>,a,<NA>
2,ASN00053009,1890-01-06,PRCP,18,<NA>,<NA>,a,<NA>
3,ASN00053010,1890-01-06,PRCP,0,<NA>,<NA>,a,<NA>
4,ASN00053027,1890-01-06,PRCP,0,<NA>,<NA>,a,<NA>
...,...,...,...,...,...,...,...,...
29567911,USC00450569,1899-12-31,SNOW,0,<NA>,<NA>,6,<NA>
29567912,USC00450872,1899-12-31,TMAX,78,<NA>,<NA>,6,<NA>
29567913,USC00450872,1899-12-31,TMIN,44,<NA>,<NA>,6,<NA>
29567914,USC00450872,1899-12-31,PRCP,25,<NA>,<NA>,6,<NA>


In [ ]:
%%time
wr.athena.read_sql_query("SELECT * FROM noaa", 
                         database="awswrangler_test", 
                         ctas_approach=False, unload_approach=True, 
                         s3_output=f"s3://{bucket}/unload/"
)

### Batching (Good for restricted memory environments)

In [ ]:
%%time

dfs = wr.athena.read_sql_query(
    "SELECT * FROM noaa",
    database="awswrangler_test",
    chunksize=True  # Chunksize calculated automatically for ctas_approach.
)

for df in dfs:  # Batching
    print(len(df.index))

In [ ]:
%%time

dfs = wr.athena.read_sql_query(
    "SELECT * FROM noaa",
    database="awswrangler_test",
    chunksize=100_000_000
)

for df in dfs:  # Batching
    print(len(df.index))